In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from numpy import arange,linspace,meshgrid,array,log,argsort
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracy,Adam
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 1.6346151294983178 32.69230258996635
Capacitance Bound(GHz): 0.38740458615415185 38.740458615415186
Shunt Bound(GHz): 3.8740458615415188 154.96183446166074
Junction Bound(GHz): 5.0 25.0


In [3]:
basis = {'O':[4,4,4],'J':[5,5],'I':[]} ; rep = 'K'
flux_point = ['La','Lc']
circuit = models.prismon(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,symmetry=True,
                         _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [4]:
lossFunction = lossDegeneracy

In [5]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)}]

In [6]:
Algo = ['Nelder-Mead','LBFGS']

In [7]:
def multiInitSearch(optimizer,initials,subspace):
    NelMea,LBFGS = [],[]
    for index,parameter in enumerate(initials):
        print(optimizer.circuitState())
        print(index,parameter)
        optimizer.circuit.initialization(parameter)  
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        LBFGS.append(optimizer.minimization(lossFunction,flux_profile,
                                                   method='L-BFGS-B',options=dict(ftol=1e-6,maxiter=5)))

        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        NelMea.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                                 method='Nelder-Mead',options=dict(fatol=1e-6,maxiter=30)))

#         optimizer.circuit.initialization(parameter)
#         optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
#         lrBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations,lr=.0005))        

    return NelMea,LBFGS       

In [8]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [9]:
def analysisPlotting(Optimization):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        #for idx in indices:
        #    paths[algo+'-'+str(idx)] = Paths[idx]
        losse[algo] = Loss
    return paths,losse

In [10]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

In [11]:
N = 5; subspace = ['La','Ca','Ja','CJa']
#initials = uniformParameters(circuit,subspace,N)
initials = truncNormalParameters(circuit,subspace,N=20,var=1)
len(initials)

20

In [12]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
Optimization = multiInitSearch(optimizer,initials,subspace)
NelMea,LBFGS = Optimization

{'La': 10.0, 'Ca': 5.0, 'Ja': 10.0, 'CJa': 50.0, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 10.0, 'Cc': 5.0, 'Jc': 10.0, 'CJc': 50.0}
0 {'La': 11.585477703743443, 'Ca': 5.112252166435358, 'Ja': 10.77471717049136, 'CJa': 50.125779370836035, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 10.0, 'Cc': 5.0, 'Jc': 10.0, 'CJc': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -9.79973D-01    |proj g|=  6.93156D-01

At iterate    1    f= -1.15459D+00    |proj g|=  6.88453D-01
  ys=-1.980E-02  -gs= 8.695E-01 BFGS update SKIPPED

At iterate    2    f= -1.73279D+00    |proj g|=  6.75426D-01

At iterate    3    f= -6.04388D+00    |proj g|=  1.36027D+01

At iterate    4    f= -6.09005D+00    |proj g|=  1.15424D+01

At iterate    5    f= -6.10683D+00    |proj g|=  1.42420D+01

           * * *

Tit   = total number of iterations
Tnf   = 

/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:549: RuntimeWarning:

Method Nelder-Mead does not use gradient information (jac).

/Users/chishti/JupyterHub/DiSuQ/Torch/optimization.py:271: RuntimeWarning:

Maximum number of iterations has been exceeded.



{'La': 5.047419548034668, 'Ca': 8.53599739074707, 'Ja': 13.720067977905273, 'CJa': 26.5004825592041, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 5.047419548034668, 'Cc': 8.53599739074707, 'Jc': 13.720067977905273, 'CJc': 26.5004825592041}
1 {'La': 10.080458719932107, 'Ca': 5.279536009391793, 'Ja': 9.9876193733373, 'CJa': 48.75270905613267, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 10.0, 'Cc': 5.0, 'Jc': 10.0, 'CJc': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.02121D+00    |proj g|=  7.10880D-01

At iterate    1    f= -1.21552D+00    |proj g|=  7.01133D-01
  ys=-3.230E-02  -gs= 9.550E-01 BFGS update SKIPPED

At iterate    2    f= -1.90490D+00    |proj g|=  9.45944D-01


/Users/chishti/JupyterHub/DiSuQ/Torch/components.py:67: RuntimeWarning:

divide by zero encountered in scalar divide




At iterate    3    f= -6.28253D+00    |proj g|=  3.64679D+01
  ys=-5.082E+02  -gs= 6.205E+00 BFGS update SKIPPED

At iterate    4    f= -6.49855D+00    |proj g|=  2.96957D+01

At iterate    5    f= -6.60290D+00    |proj g|=  3.10546D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      5     25     11     2     0   3.105D+01  -6.603D+00
  F =  -6.6029047966003418     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
{'La': 4.344690322875977, 'Ca': 8.36350154876709, 'Ja': 12.09453010559082, 'CJa': 28.177677154541016, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 4.344690322875977, 'Cc': 8.36


At iterate    2    f= -1.87962D+00    |proj g|=  7.65187D-01

At iterate    3    f= -5.26140D+00    |proj g|=  2.52568D+01

At iterate    4    f= -6.07792D+00    |proj g|=  2.75339D+00

At iterate    5    f= -6.09379D+00    |proj g|=  1.97534D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      5     21      5     1     0   1.975D+00  -6.094D+00
  F =  -6.0937895774841309     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
{'La': 5.705448150634766, 'Ca': 9.025986671447754, 'Ja': 10.368412017822266, 'CJa': 25.71257209777832, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 5.705448150634766,


At iterate    2    f= -1.92217D+00    |proj g|=  9.86345D-01

At iterate    3    f= -5.71010D+00    |proj g|=  2.97576D+01

At iterate    4    f= -1.04628D+01    |proj g|=  4.49992D+01

At iterate    5    f= -1.05221D+01    |proj g|=  4.49992D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      5     34      5     1     0   4.500D+01  -1.052D+01
  F =  -10.522086143493652     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
{'La': 4.328405380249023, 'Ca': 8.424405097961426, 'Ja': 14.404072761535645, 'CJa': 29.315933227539062, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 4.328405380249023

In [ ]:
iterations = 30

In [ ]:
plotCompare(arange(iterations),{'Nelder-Mead':NelMea[0][0]['D'],'LBFGS':LBFGS[0][0]['D']},'Degeneracy','iterations',export='pdf',size=(600,800))

In [ ]:
plotCompare(arange(iterations),{'Nelder-Mead':NelMea[0][0]['E10'],'LBFGS':LBFGS[0][0]['E10']},'Ist Excitation','iterations',export='pdf',size=(600,800))

In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in Optimization]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution',export='pdf',size=(600,800))

### spectrum - optimal circuits

In [ ]:
Nx,Ny  = 51,1
PhiX = linspace(0,1,Nx,endpoint=True)
PhiY = linspace(0,1,Ny,endpoint=True)
flux_range = meshgrid(PhiX,PhiY)
flux_range = array((flux_range[0].flatten(),flux_range[1].flatten())).T
flux_manifold = [flux for flux in tensor(flux_range)]

In [ ]:
Degeneracy,plots = dict(),dict()
for algo,path in paths.items():
    parameters = dict(zip(subspace,path[-1]))
    static.update(parameters)
    circuit.initialization(static)
    H_LC = circuit.chargeHamiltonianLC()
    H_J = circuit.josephsonCharge
    E0,(Ex1,Ex2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=True)
    Ex2 = Ex2.detach().numpy()
    Ex1 = Ex1.detach().numpy()
    degeneracy = log(Ex2/Ex1)
    Degeneracy[algo]=degeneracy
    plots[algo+'-E10']=Ex1
    plots[algo+'-E20']=Ex2
    
#     optimizer.circuit.initialization(static)
#     Spectrum = [optimizer.spectrumOrdered(flux) for flux in flux_profile]
#     spectrum = stack([spectrum[:3] for spectrum,state in Spectrum]).detach().numpy()
#     Ex1 = spectrum[:,1]-spectrum[:,0]
#     Ex2 = spectrum[:,2]-spectrum[:,0]
#     plots[algo+'-e10']=Ex1
#     plots[algo+'-e20']=Ex2

In [ ]:
plotCompare(flux_range[:,0],Degeneracy,'Degeneracy-Optimal Circuits','flux profile',export='pdf',size=(600,800))

In [ ]:
plotCompare(flux_range[:,0],plots,'Spectrum-Optimal Circuits','flux profile','energy(GHz)',export='pdf',size=(600,800))

In [ ]:
subspace

In [ ]:
paths['Nelder-Mead'][-1]

In [ ]:
paths['LBFGS'][-1]

### Charge-Flux susceptibility

### Wavefunction